<a href="https://colab.research.google.com/github/Chan-Sicpama/gcs/blob/main/N_BEATS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 실습 데이터셋 준비
1. 런타임을 GPU로 변경해주세요.
2. continuous dataset.csv 를 업로드해주세요.

In [ ]:
from google.colab import files
files.upload()

# N-BEATS
N-BEATS: Neural basis expansion analysis for interpretable time series forecasting, Boris et al., ICLR 2020


- N-BEATS는 M4 competition dataset 에 뛰어난 성능을 보였습니다.
- 시계열 분해(time-series decomposition)의 개념과 비슷하게, 딥러닝 기반의 예측 결과값을 추세(trend), 계절성(seasonality), 잔차(residual) 로 나누어 예측합니다. 이를 통해 모델의 해석가능성을 끌어올릴 수 있습니다.
- 추세와 계절성은 입력 데이터(backcast)에서 나타나는 정보를 이용하여 얻어내며, 얻어진 정보를 통해 미래 데이터(forecast)를 예측합니다. 


![n-beats.png](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbfQXYg%2FbtqH2y7ZIaw%2F5PURFStjssBABVi31dxzX0%2Fimg.png)

In [ ]:
import time
import numpy as np
import pandas as pd

import torch
from torch import nn

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

In [ ]:
class NBeatsNet(nn.Module):
    def __init__(self,
                 forecast_length=24,
                 backcast_length=96,
                 theta_dim=(2, 8, 3),
                 n_trend=3,
                 n_seasonality=3,
                 n_residual=3,
                 hidden_dim=512):
        super(NBeatsNet, self).__init__()        
            
        self.trend_stack = []
        self.seasonality_stack = []
        self.residual_stack = []
        
        for i in range(n_trend):
            self.trend_stack.append(Block(backcast_length, forecast_length, hidden_dim, theta_dim[0], mode='trend'))    

        for i in range(n_seasonality):
            self.seasonality_stack.append(Block(backcast_length, forecast_length, hidden_dim, theta_dim[1], mode='seasonality'))
        
        for i in range(n_residual):
            self.residual_stack.append(Block(backcast_length, forecast_length, hidden_dim, theta_dim[2], mode='residual'))
        
        
        self.trend_stack = nn.ModuleList(self.trend_stack)
        self.seasonality_stack = nn.ModuleList(self.seasonality_stack)
        self.residual_stack = nn.ModuleList(self.residual_stack)

    def forward(self, backcast):
        backcast_stack = []
        forecast_stack = []

        for layer in self.trend_stack:
            b, f = layer(backcast)
            backcast_stack.append(b)
            forecast_stack.append(f)
            backcast = backcast - b

        for layer in self.seasonality_stack:
            b, f = layer(backcast)
            backcast_stack.append(b)
            forecast_stack.append(f)
            backcast = backcast - b


        for layer in self.residual_stack:
            b, f = layer(backcast)
            backcast_stack.append(b)
            forecast_stack.append(f)
            backcast = backcast - b

        backcast = torch.stack(backcast_stack, 0)
        forecast = torch.stack(forecast_stack, 0)
        return backcast, forecast


# ours
class Block(nn.Module):
    def __init__(self, backcast_length, forecast_length, hidden_dim, theta_dim, mode):
        super(Block, self).__init__()
        self.backcast_length = backcast_length
        self.forecast_length = forecast_length
        self.theta_dim = theta_dim
        self.mode = mode

        self.MLP = nn.Sequential(
                nn.Linear(backcast_length, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, hidden_dim),
                nn.ReLU()
            )

        if mode == 'residual':
            # residual
            self.theta_b = nn.Linear(hidden_dim, theta_dim)
            self.theta_f = nn.Linear(hidden_dim, theta_dim)
            self.linear_b = nn.Linear(theta_dim, backcast_length)
            self.linear_f = nn.Linear(theta_dim, forecast_length)
        else:
            # trend, seasonality
            self.theta = nn.Linear(hidden_dim, theta_dim)


    def forward(self, x):
        # x: (batch size, backcast length)
        t_b = ((torch.arange(start=0, end=self.backcast_length, device=x.device, dtype=torch.float) - self.backcast_length) / 24 ) # (backcast_len,)
        t_f = ((torch.arange(start=0, end=self.forecast_length, device=x.device, dtype=torch.float)) / 24 ) # (forecast_len,)

        # x -> h
        x = self.MLP(x)

        # h -> theta -> backcast, forecast
        if self.mode == 'residual':
            # residual block
            theta_b = self.theta_b(x)
            theta_f = self.theta_f(x)
            b = self.linear_b(theta_b) # (batch, backcast length)
            f = self.linear_f(theta_f) # (batch, forecast length)
        else:
            theta_b = self.theta(x)
            theta_f = theta_b

            if self.mode == 'trend':
                b = self.get_trend(theta_b, t_b)
                f = self.get_trend(theta_f, t_f)
                
            elif self.mode == 'seasonality':
                b = self.get_seasonality(theta_b, t_b)
                f = self.get_seasonality(theta_f, t_f)
        return b, f

    def get_trend(self, theta, t):
        # theta dim = 0 -> 수평선 (y=a)
        # theta dim = 1 -> 직선 (y=bx)
        # theta dim = 2 -> 이차곡선 (y=cx^2)
        # ...
        T = torch.stack([t ** i for i in range(theta.shape[1])])  # (theta_dim ,sequence length)
        out = torch.einsum('bt,ts->bs', theta, T)
        return out


    def get_seasonality(self, theta, t):
        s1 = torch.stack([torch.cos(2 * np.pi * (i+1) * t) for i in range(self.theta_dim//2)]).float()  # H/2-1
        s2 = torch.stack([torch.sin(2 * np.pi * (i+1) * t) for i in range(self.theta_dim//2)]).float()
        S = torch.cat([s1, s2])
        out = torch.einsum('bt,ts->bs', theta, S)
        return out

In [ ]:
# model hyper parameters
forecast_length=24
backcast_length=96
theta_dim=(2, 8, 3)
n_trend=3
n_seasonality=3
n_residual=3
hidden_dim=64

In [ ]:
model = NBeatsNet(forecast_length, backcast_length, theta_dim, n_trend, n_seasonality, n_residual, hidden_dim)

In [ ]:
model

In [ ]:
inputs = torch.randn(3,backcast_length)
labels = torch.randn(3,forecast_length)

In [ ]:
backcast, forecast = model(inputs)

In [ ]:
backcast.shape, forecast.shape

In [ ]:
def plot(inputs, labels, backcast, forecast, n_trend=3, n_seasonality=3, n_residual=3, batch_idx=0):
    backcast_length = backcast.shape[-1]
    forecast_length = forecast.shape[-1]
    
    x = inputs.data.cpu()[batch_idx]
    y = labels.data.cpu()[batch_idx]
    b = backcast.data.cpu()[:,batch_idx]
    f = forecast.data.cpu()[:,batch_idx]

    backcast_trend = b[:n_trend].sum(0)
    forecast_trend = f[:n_trend].sum(0)
    backcast_seasonality = b[n_trend:n_trend+n_seasonality].sum(0)
    forecast_seasonality = f[n_trend:n_trend+n_seasonality].sum(0)
    backcast_residual = b[n_trend+n_seasonality:].sum(0)
    forecast_residual = f[n_trend+n_seasonality:].sum(0)
    backcast_final = b.sum(0)
    forecast_final = f.sum(0)

    plt.figure(figsize=(20,8))
    plt.gca().axvspan(0, backcast_length, alpha=0.3, color='gray')
    forecast_range = range(backcast_length, backcast_length+forecast_length)

    plt.plot(x, c='black')
    plt.plot(forecast_range, y, c='black')
    plt.plot(backcast_trend, c='tab:blue')
    plt.plot(forecast_range, forecast_trend, c='tab:blue')

    plt.plot(x+5, c='black')
    plt.plot(forecast_range, y+5, c='black')
    plt.plot(backcast_seasonality+5, c='tab:orange')
    plt.plot(forecast_range, forecast_seasonality+5, c='tab:orange')

    plt.plot(x+10, c='black')
    plt.plot(forecast_range, y+10, c='black')
    plt.plot(backcast_residual+10, c='tab:green')
    plt.plot(forecast_range, forecast_residual+10, c='tab:green')
    
    plt.plot(x+15, c='black')
    plt.plot(forecast_range, y+15, c='black')
    plt.plot(backcast_final+15, c='tab:red')
    plt.plot(forecast_range, forecast_final+15, c='tab:red')    
    
    
    
    legend_elements =[Line2D([0], [0], color='black', lw=4, label='Line'),
                      Line2D([0], [0], color='tab:blue', lw=4, label='Line'),
                      Line2D([0], [0], color='tab:orange', lw=4, label='Line'),
                      Line2D([0], [0], color='tab:green', lw=4, label='Line'),
                      Line2D([0], [0], color='tab:red', lw=4, label='Line')]
    plt.gca().legend(legend_elements, ['Real values', 'Trend', 'Seasonality', 'Residual', 'Final prediction'], loc='lower left', fontsize=15)

    plt.show()

In [ ]:
plot(inputs, labels, backcast, forecast, batch_idx=0)

# Electricity Load Forecasting dataset

https://www.kaggle.com/saurabhshahane/electricity-load-forecasting

In [ ]:
df = pd.read_csv('continuous dataset.csv')

In [ ]:
df = df[['datetime','nat_demand']]

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(df['nat_demand'][:300])
plt.show()

In [ ]:
train_df = df[df['datetime'].map(lambda x:x.split('-')[0] in ['2015', '2016', '2017', '2018'])]
val_df = df[df['datetime'].map(lambda x:x.split('-')[0] in ['2019'])]
test_df = df[df['datetime'].map(lambda x:x.split('-')[0] in ['2020'])]

In [ ]:
train_df.shape, val_df.shape, test_df.shape

In [ ]:
train_df.head()

In [ ]:
val_df.head()

In [ ]:
test_df.head()

In [ ]:
class StandardScaler():
    def __init__(self):
        self.mean = None
        self.stdev = None
        
    def fit(self, x):
        if not torch.is_tensor(x):
            x = torch.tensor(x)
        self.mean = x.mean()
        self.stdev = x.std()
        
    def scale(self, x):
        if not torch.is_tensor(x):
            x = torch.tensor(x)
        return (x - self.mean.to(x.device)) / self.stdev.to(x.device)
    
    def unscale(self, x):
        if not torch.is_tensor(x):
            x = torch.tensor(x)
        return x * self.stdev.to(x.device) + self.mean.to(x.device)
        
class TimeseriesDataset(torch.utils.data.Dataset):   
    def __init__(self, data, backcast_length, forecast_length, scaler=None):
        self.data_datetime = data['datetime'].values
        self.data = data['nat_demand'].values
        self.backcast_length = backcast_length
        self.forecast_length = forecast_length
        
        if scaler is None:
            # train
            self.scaler = StandardScaler()
            self.scaler.fit(self.data)
        else:
            self.scaler = scaler
        self.data = self.scaler.scale(self.data)
        

    def __len__(self):
        return len(self.data) - self.backcast_length - self.forecast_length + 1

    def __getitem__(self, idx):
        x_end = idx + self.backcast_length
        y_end = x_end + self.forecast_length

        x = self.data[idx:x_end]
        y = self.data[x_end:y_end]
        
        x = x.float()
        y = y.float()
        
        return x, y 
    
    def unscale(self, x):
        return self.scaler.unscale(x)

In [ ]:
batch_size = 128
num_workers = 2
pin_memory = True
learning_rate = 1e-3

In [ ]:
train_dataset = TimeseriesDataset(train_df, backcast_length, forecast_length)
val_dataset = TimeseriesDataset(val_df, backcast_length, forecast_length, scaler=train_dataset.scaler)
test_dataset = TimeseriesDataset(test_df, backcast_length, forecast_length, scaler=train_dataset.scaler)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                                               num_workers=num_workers, pin_memory=pin_memory)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, 
                                               num_workers=num_workers, pin_memory=pin_memory)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, 
                                               num_workers=num_workers, pin_memory=pin_memory)

In [ ]:
model = NBeatsNet()
model = model.cuda()

criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
num_epoch = 10

In [ ]:
start_time = time.time()
best_model_state_dict = None
best_val_loss = 10000
for epoch in range(num_epoch):
    train_loss_mean = 0
    val_loss_mean = 0
    
    # train
    model.train()
    for i, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.cuda()
        labels = labels.cuda()
        
        backcast, forecast = model(inputs)
        loss = criterion(backcast.sum(0), inputs) + criterion(forecast.sum(0), labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss_mean += loss.item()
        
        if i % 100 == 0:
            print('epoch [{}/{}] iter [{:03d}/{:03d}] loss [{:.4f}] elapsed time [{:.2f}min]'.format(epoch, num_epoch, i, len(train_dataloader), loss.item(), (time.time()-start_time)/60))
    
    train_loss_mean = train_loss_mean/len(train_dataloader)
    
    # validation
    model.eval()
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(val_dataloader):
            inputs = inputs.cuda()
            labels = labels.cuda()

            backcast, forecast = model(inputs)
            loss = criterion(backcast.sum(0), inputs) + criterion(forecast.sum(0), labels)
            val_loss_mean += loss.item()

        val_loss_mean = val_loss_mean/len(val_dataloader)

        print('epoch [{}/{}] train loss [{:.4f}] validation loss [{:.4f}] elapsed time [{:.2f} min]\n'.format(epoch, num_epoch, train_loss_mean, val_loss_mean, (time.time()-start_time)/60))

        if val_loss_mean < best_val_loss:
            best_val_loss = val_loss_mean
            best_model_state_dict = model.state_dict()


In [ ]:
# load best model
model.load_state_dict(best_model_state_dict)
model.eval()

In [ ]:
def MAE(x, y):
    return nn.L1Loss()(x,y)

def RMSE(x,y):
    return (nn.MSELoss()(x,y)).sqrt()

def MAPE(x,y):
    eps = 1e-8
    return ((x-y).abs() / (y+eps)).mean()

def sMAPE(x,y):
    eps = 1e-8
    return ((x-y).abs()*2 / (x+y+eps)).mean()
    
def metric(x,y):
    return [MAE(x,y), RMSE(x,y), MAPE(x,y), sMAPE(x,y)]

In [ ]:


inputs_list = []
labels_list = []
backcast_list = []
forecast_list = []

with torch.no_grad():
    for i, (inputs, labels) in enumerate(val_dataloader):
        inputs = inputs.cuda()
        labels = labels.cuda()

        backcast, forecast = model(inputs)

        inputs_list.append(inputs.data.cpu())
        labels_list.append(labels.data.cpu())
        backcast_list.append(backcast.data.cpu())
        forecast_list.append(forecast.data.cpu())

        
    inputs = torch.cat(inputs_list)
    labels = torch.cat(labels_list)
    backcast = torch.cat(backcast_list,1)
    forecast = torch.cat(forecast_list,1)
    
    
    inputs_unscaled = train_dataset.unscale(inputs)
    labels_unscaled = train_dataset.unscale(labels)
    backcast_unscaled = train_dataset.unscale(backcast.sum(0))
    forecast_unscaled = train_dataset.unscale(forecast.sum(0))
    
    mae, rmse, mape, smape = metric(labels_unscaled, forecast_unscaled)
    
    print('MAE [{:.4f}] RMSE [{:.4f}] MAPE [{:.4f}] sMAPE [{:.4f}]'.format(mae, rmse, mape, smape))

In [ ]:
from IPython.display import clear_output 
for i in range(0, len(inputs), 3):
    clear_output(wait=True)
    plot(inputs, labels, backcast, forecast, batch_idx=i)